In [12]:
import numpy as np
import pandas as pd
import seaborn as sb
% matplotlib inline
from datetime import datetime, timedelta

from bs4 import BeautifulSoup as bs
import requests
import re
import pickle

In [2]:
rankings_p = '../data/rankings.p'
headers_p = '../data/headers.p'

# function to scrape marathon data
def scrape_marathon_data():
    years = range(2002, 2018)
    letters = [chr(x + ord('A')) for x in range(0, 26)]

    URL = "https://services.datasport.com/{}/lauf/lamara/ALFA{}.HTM"
    
    rankings = {}
    headers = {}
    for year in years:
        print(str(year) + ": ", end='')
        rankings[str(year)] = []
        for letter in letters:
            r = requests.get(URL.format(year, letter))
            soup = bs(r.text, 'html.parser')
            title = soup.find('title').getText()
            if(title != 'Adresse nicht vorhanden / The address is not available'):
                data = soup.findAll('font', {'size':'2'})

                print(letter, end='')
                headers[str(year)] = data[0].getText()
                rankings[str(year)].extend(re.compile('¦ *\r\n').split(data[1].getText()))
            else:
                print("Skipped")
        print('')

    # Pickle data
    pickle.dump(rankings, open(rankings_p, 'wb'))
    pickle.dump(headers, open(headers_p, 'wb'))
    return rankings, headers

# Only scrape if pickle unavailable
def get_data(force_scrape=False):
    if force_scrape:
        print('Scrape data from website')
        return scrape_marathon_data()
    try:
        print('Trying to load data from pickle')
        return pickle.load(open(rankings_p, 'rb')), pickle.load(open(headers_p, 'rb'))
    except (OSError, IOError) as e:
        print('Failed to load pickle:')
        print(e)
        print('Scrape data from website')
        return scrape_marathon_data()


In [3]:
"""
This function finds the position of where the pages should be split into columns
based on whitespace
"""
def parse_columns(ranking, header):
    column_structure = {}
    column_mask = np.ones(np.array(list(ranking[0])).shape, dtype=bool)
    spaces_mask = np.ones(np.array(list(ranking[0])).shape)*32
    
    # create mask of space positions for field delimiter detection
    for line in ranking[:-1]:
        cur_line = np.array(list(line)).view(np.uint32)
        
        try:
            column_mask = np.logical_and(column_mask, np.equal(cur_line, spaces_mask))
        except:
            #print("Skipped line:", line)
            #print(cur_line.shape, column_mask.shape)
            pass
        
    # include header in space position mask
    header_int = np.array(list(header[:column_mask.shape[0]])).view(np.uint32)
    column_mask = np.logical_and(column_mask, np.equal(header_int, spaces_mask))
        
    # find field delimiter positions
    inside_space_col = False
    delimiters = []
    for i, is_space_col in enumerate(list(column_mask)):
        if is_space_col:
            if not inside_space_col:
                inside_space_col = True
                delimiters.append(i)
                
        elif inside_space_col:
            inside_space_col = False
            
    return delimiters
    

"""
Seperates the lists of lines into columns and create a dictionary of dataframes
"""
def separate_cols(rankings, headers):
    # dictionary to keep dataframes in
    dfs = {}
    for year in rankings:
        ranking = rankings[year]
        header = headers[year]
        
        # fix offset headers in 2007-2009 data
        if year in ['2007', '2008', '2009']:
            header = header.replace('an lieu', 'an   lieu')
        delimiters = parse_columns(ranking, header)

        # find positions for each field
        start = 0
        cols = {} 
        unnamed = 0
        for delimiter in delimiters:
            key = header[start:delimiter+1].strip()
            if key == '':
                key = unnamed
                unnamed += 1
            cols[key] = (start, delimiter+1)
            start = delimiter+1

        start = 0
        result = {}
        for i, line in enumerate(ranking):
            if '\r\ntotal ' not in line and '\r\nTotal' not in line:
                for col in cols:
                    start, end = cols[col]
                    if col in result:
                        result[col].append(line[start:end].strip())
                    else:
                        result[col] = [line[start:end].strip()]
        dfs[year] = pd.DataFrame(result, columns=result.keys()) 
    print('Columns separated')
    return dfs

In [4]:
mapping = {
    'lieu': '',
    0: 'DELETE',
    2: 'DELETE',
    3: 'DELETE',
    'catégorie': '',
    'nom': '',
    1: 'DELETE',
    'temps': '',
    'Jg': 'an',
    'an lieu': '',
    'overall': '',
    'retard': '',
    'équipe': '',
    'moyenne': '',
    'Rang': 'rang',
    'nom/lieu': '',
    'Stnr': 'doss',
    'an': '',
    'pénalité': '',
    'doss': '',
    'temps net': '',
    'rang': '',
    '¦': 'DELETE',
    'équipe/lieu': '',
    'Rückstand': 'retard',
    'Kategorie': 'catégorie',
    'Team/Ortschaft': 'équipe/lieu',
    'Name/Ort': 'nom/lieu',
    'Zeit': 'temps'
}

In [5]:
"""
Where possible, this function matches the names of the columns 
of the individual dataframes
"""
def normalize_column_names(dfs_):
    dfs_c = dfs_.copy()
    for year, _ in dfs_c.items():
        for header_name in list(dfs_c[year]):
            if mapping[header_name] == 'DELETE':
                dfs_c[year] = dfs_c[year].drop(header_name, axis=1)
            elif mapping[header_name] != "":
                dfs_c[year] = dfs_c[year].rename(columns = {header_name : mapping[header_name]})
                print("Renamed {} to {} in year {}".format(
                    header_name, mapping[header_name], year))
    print('Column names normalized')
    return dfs_c
                


def print_col_overview(dfs):
    # fields present in all years
    shared_fields = set.intersection(*[set(v) for k, v in dfs.items()])
    print("Shared fields:", shared_fields)

    fields = set()
    for year in sorted(dfs):
        fields = fields.union(set(list(dfs[year])))
        print(year)
        print(set(dfs[year]).difference(shared_fields))
    print("Union remaining fields:", fields.difference(shared_fields))

In [6]:
def fix_cities(dfs_):
    dfs_c = dfs_.copy()
    """
    Years 2002-2006 have a team/city field and a name/city field.
    If a team is provided, the city is added to the end of the name,
    after a comma (',').
    
    This function removes the teams and converts the column 'équipe/lieu' 
    to a dedicated 'lieu' column
    """
    for year, df in dfs_c.items():
        col_name = 'nom/lieu'
        col_team = 'équipe/lieu'
        if col_name in df:
            for i, row in df.iterrows():
                m = re.compile('(.*), (.*)').match(row[col_name])
                if m is not None:
                    dfs_c[year].at[i, col_name] = m.group(1)
                    dfs_c[year].at[i, col_team] = m.group(2)
            # rename columns
        rename_dict = {col_name : 'nom', col_team: 'lieu'}
        dfs_c[year].rename(columns=rename_dict, inplace=True)
    print('Cities fixed')
    return dfs_c

In [7]:
"""
Returns the fields that are present in all years
"""
def get_shared_fields(dfs):
    return set.intersection(*[set(v) for k, v in dfs.items()])

"""
Returns the fields that are only present in some of the years
"""
def get_unshared_fields(dfs):
    return set.union(*[set(v) for k, v in dfs.items()]).difference(get_shared_fields(dfs))

"""
Deletes columns from dataframes that are only present in some of the years
"""
def drop_uncommon_fields(dfs_):
    dfs_c = dfs_.copy()
    for year, df in dfs_c.items():
        dfs_c[year] = dfs_c[year][list(get_shared_fields(dfs_c))]
    print('Uncommon fields dropped')
    return dfs_c

In [8]:
def create_year_column(dfs_):
    dfs_c = dfs_.copy()
    for year in dfs_c:
        df = dfs_c[year].copy()
        df['race-year'] = int(year)
        dfs_c[year] = df
    print('Year column created')
    return dfs_c

In [9]:
def create_merged_df(dfs_):
    return pd.concat(list(dfs_.values()))

In [10]:
def normalize_categories(df_):
    """
    rename important categories and drop unused
    """
    df_c = df_.copy()
    df_c['catégorie'] = df_['catégorie'].replace({
        r'^(\d\d)[-/][MQS]*([DH])\d+$' : r'\1|\2',
        r'^Q([DH])\d$' : r'10|\1',
        r'^S([DH])\d$' : r'21|\1',
        r'^M([DH])\d$' : r'42|\1'
    }, regex = True)
    
    # columns to keep
    keep = [str(x) + '|H' for x in [10, 21, 42]]
    keep.extend([str(x) + '|D' for x in [10, 21, 42]])
    
    # drop all other rows
    df_c = df_c[df_c['catégorie'].isin(keep)]
    
    # split catégorie into two columns
    df_c[['distance', 'gender']] = df_c['catégorie'].str.split(
        '|', expand=True
    )
    
    # replace german gender abbreviations
    df_c['gender'] = df_c['gender'].replace({
        'H' : 'male',
        'D' : 'female'
    })
    
    df_c = df_c.drop('catégorie', axis=1)
    
    return df_c
    

In [11]:
# Get the data
rankings, headers = get_data()
# Separate data into columns and build dataframes
dfs = separate_cols(rankings, headers)
# Make sure the same columns have the same column names
dfs_n = normalize_column_names(dfs)
# Fix cities which are in the name column in some cases
dfs_f = fix_cities(dfs_n)
# Drop columns that are unique to some years
dfs_d = drop_uncommon_fields(dfs_f)
# Create year columns
dfs_y = create_year_column(dfs_d)

Trying to load data from pickle
Columns separated
Renamed Zeit to temps in year 2002
Renamed Name/Ort to nom/lieu in year 2002
Renamed Jg to an in year 2002
Renamed Kategorie to catégorie in year 2002
Renamed Rückstand to retard in year 2002
Renamed Rang to rang in year 2002
Renamed Team/Ortschaft to équipe/lieu in year 2002
Renamed Stnr to doss in year 2002
Column names normalized
Cities fixed
Uncommon fields dropped
Year column created


In [15]:
# Create final dataframe
df = create_merged_df(dfs_y)
# normalize categories
df = normalize_categories(df)

In [16]:
# Print overview of columns for inspection
df.head()

,nom,an,lieu,doss,rang,temps,retard,race-year,distance,gender
0,Abantes Jose,55,Sierre,(3001),1.,"1:09.47,4",-----,2003,21,male
1,Abbiati Franco,65,Umiken,(1965),474.,"4:56.06,0","2:42.53,7",2003,42,male
2,Abcottspon Julia,50,Gwatt (Thun),(3002),4.,"1:37.53,9","11.53,9",2003,21,female
3,Abegglen Eddy,54,Mürren,(1940),432.,"4:24.11,4","1:47.58,3",2003,42,male
4,Abel Marit,55,N-Hamar,(7001),112.,"58.03,1","16.32,2",2003,10,female


In [18]:
# Save dataframe to csv
df.to_csv('../data/marathon-data.csv', sep='\t')

In [54]:
df_loc = pd.read_csv('../data/marathon-data-with-geo.csv')

In [55]:
merge_cols = df_loc[[
    'lieu', 'format_city', 'Canton', 
    'country', 'lat', 'lng'
]].drop_duplicates()

In [72]:
df_merged = pd.merge(df, merge_cols, on='lieu', how='inner')
df_merged = df_merged.dropna(subset=['country'])
df_merged.to_csv('../data/marathon-data-gender-race-loc.csv')

In [13]:
df_data = pd.read_csv('marathon-data-gender-race-loc.csv')

/Users/xiangzhemeng/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
df_data

,Unnamed: 0,nom,an,lieu,doss,rang,temps,retard,race-year,distance,gender,format_city,Canton,country,lat,lng
0,0,Abantes Jose,55,Sierre,(3001),1.,"1:09.47,4",-----,2003,21,male,Sierre,VS,CH,46.294131,7.533536
1,1,Epiney Luc,45,Sierre,(629),132.,"3:56.53,1","1:06.41,1",2003,42,male,Sierre,VS,CH,46.294131,7.533536
2,2,Giachino Primo,51,Sierre,(756),206.,"4:31.31,2","1:41.19,2",2003,42,male,Sierre,VS,CH,46.294131,7.533536
3,3,Mabire Cedric,73,Sierre,(1154),497.,"1:41.40,2",La Tour-de-,2003,42,male,Sierre,VS,CH,46.294131,7.533536
4,4,Matter Frank,72,Sierre,(4558),76.,"1:26.20,3","19.44,8",2003,21,male,Sierre,VS,CH,46.294131,7.533536
5,5,Seara Ricardo,79,Sierre,(8418),11.,"33.33,1","2.29,8",2003,10,male,Sierre,VS,CH,46.294131,7.533536
6,6,Theytaz Jean-Pierre,83,Sierre,(8512),12.,"33.53,1","2.49,8",2003,10,male,Sierre,VS,CH,46.294131,7.533536
7,7,Zufferey Christian,65,Sierre,(5630),537.,"1:56.23,5","49.48,0",2003,21,male,Sierre,VS,CH,46.294131,7.533536
8,8,Zufferey Jean-Claude,55,Sierre,(5631),192.,"1:39.55,9","30.08,5",2003,21,male,Sierre,VS,CH,46.294131,7.533536
9,9,Zufferey Julie,87,Sierre,(8655),141.,"56.42,4","24.14,6",2003,10,female,Sierre,VS,CH,46.294131,7.533536


In [16]:
df_data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [25]:
df_data.drop(df_data.loc[df_data['an'] == '????'].index, axis=0,inplace=True)

In [27]:
df_data.drop(df_data.loc[df_data['an'] == '??'].index, axis=0,inplace=True)

In [30]:
df_data['an'] = df_data['an'].astype(int)

In [35]:
df_data['an'] = df_data['an'].apply(lambda x: 1900+x if x < 100 else x)

In [37]:
df_data.drop(['lieu','doss','retard'], axis=1, inplace=True)

In [39]:
df_data.columns = ['name','birth year','rank','time','race year','category','gender','city','canton','country','latitude','longitude']

In [44]:
df_data.drop(df_data[df_data['city'].isnull() == True].index, axis=0, inplace=True)

In [47]:
df_data['age'] = 2017 - df_data['birth year']

In [49]:
def ageGroup(x):
    if x >= 20 and x < 30:
        return '20-30'
    elif x >= 30 and x < 40:
        return '30-40'
    elif x >= 40 and x < 50: 
        return '40-50'
    else:
        return 'other'

In [53]:
df_data['age group'] = df_data['age'].apply(ageGroup)

In [63]:
df_data['canton'] = df_data['canton'].fillna('')

In [64]:
df_data.isnull().sum()

name          0
birth year    0
rank          0
time          0
race year     0
category      0
gender        0
city          0
canton        0
country       0
latitude      0
longitude     0
age           0
age group     0
dtype: int64

In [65]:
df_data

,name,birth year,rank,time,race year,category,gender,city,canton,country,latitude,longitude,age,age group
0,Abantes Jose,1955,1.,"1:09.47,4",2003,21,male,Sierre,VS,CH,46.294131,7.533536,62,other
1,Epiney Luc,1945,132.,"3:56.53,1",2003,42,male,Sierre,VS,CH,46.294131,7.533536,72,other
2,Giachino Primo,1951,206.,"4:31.31,2",2003,42,male,Sierre,VS,CH,46.294131,7.533536,66,other
3,Mabire Cedric,1973,497.,"1:41.40,2",2003,42,male,Sierre,VS,CH,46.294131,7.533536,44,40-50
4,Matter Frank,1972,76.,"1:26.20,3",2003,21,male,Sierre,VS,CH,46.294131,7.533536,45,40-50
5,Seara Ricardo,1979,11.,"33.33,1",2003,10,male,Sierre,VS,CH,46.294131,7.533536,38,30-40
6,Theytaz Jean-Pierre,1983,12.,"33.53,1",2003,10,male,Sierre,VS,CH,46.294131,7.533536,34,30-40
7,Zufferey Christian,1965,537.,"1:56.23,5",2003,21,male,Sierre,VS,CH,46.294131,7.533536,52,other
8,Zufferey Jean-Claude,1955,192.,"1:39.55,9",2003,21,male,Sierre,VS,CH,46.294131,7.533536,62,other
9,Zufferey Julie,1987,141.,"56.42,4",2003,10,female,Sierre,VS,CH,46.294131,7.533536,30,30-40


In [66]:
df_data = df_data[['name','birth year','age','age group','gender','race year','category','rank','time','city','canton','country','latitude','longitude']]

In [67]:
df_data

,name,birth year,age,age group,gender,race year,category,rank,time,city,canton,country,latitude,longitude
0,Abantes Jose,1955,62,other,male,2003,21,1.,"1:09.47,4",Sierre,VS,CH,46.294131,7.533536
1,Epiney Luc,1945,72,other,male,2003,42,132.,"3:56.53,1",Sierre,VS,CH,46.294131,7.533536
2,Giachino Primo,1951,66,other,male,2003,42,206.,"4:31.31,2",Sierre,VS,CH,46.294131,7.533536
3,Mabire Cedric,1973,44,40-50,male,2003,42,497.,"1:41.40,2",Sierre,VS,CH,46.294131,7.533536
4,Matter Frank,1972,45,40-50,male,2003,21,76.,"1:26.20,3",Sierre,VS,CH,46.294131,7.533536
5,Seara Ricardo,1979,38,30-40,male,2003,10,11.,"33.33,1",Sierre,VS,CH,46.294131,7.533536
6,Theytaz Jean-Pierre,1983,34,30-40,male,2003,10,12.,"33.53,1",Sierre,VS,CH,46.294131,7.533536
7,Zufferey Christian,1965,52,other,male,2003,21,537.,"1:56.23,5",Sierre,VS,CH,46.294131,7.533536
8,Zufferey Jean-Claude,1955,62,other,male,2003,21,192.,"1:39.55,9",Sierre,VS,CH,46.294131,7.533536
9,Zufferey Julie,1987,30,30-40,female,2003,10,141.,"56.42,4",Sierre,VS,CH,46.294131,7.533536


In [69]:
df_data.to_csv('marathon_data.csv')